# Load dataset

In [1]:
#Acknowlegement: Some codes used to initialise BERT are from BERT tutorial(https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP)

In [ ]:
# Import necessary libraries
import re
import string
import os 
from collections import defaultdict
from collections import Counter
import time
import numpy as np
import torch
import datetime
import copy
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from covid_tools import *
from BERTs import *

In [ ]:
# load orginal data(COVID-19 TREC-IS 2020 task) from json file.
import json
import pandas as pd
cv19_dc = pd.read_json('./Pythonbooks/Data/COVID/cv19_dc_test.json',lines=True,orient='records')
cv19_ws = pd.read_json('./Pythonbooks/Data/COVID/cv19_washington_state_test.json',lines=True,orient='records')
cv19_ny = pd.read_json('./Pythonbooks/Data/COVID/cv19_nyc_test.json',lines=True,orient='records')

# Read lablled tweets from json file which contains all labels based on union of labels from all human assessors.
all_labels  = read_union_labels_from_file()

# Merge original dataset with lablled dataset based on id of tweets
cv19_dc_labeled = pd.merge(cv19_dc,all_labels,on=['id'])
cv19_ws_labeled = pd.merge(cv19_ws,all_labels,on=['id'])
cv19_ny_labeled = pd.merge(cv19_ny,all_labels,on=['id'])

# As for priorization task, we use majority vote of labels instead of union.
'''More specifically, if one tweet is assessed by more than two assessors,
 we use the most common label as the priority label. And if one tweet is assessed by less than two assessors, 
 we randomly choose one label as the true label. 
 '''
cv19_dc_labeled = extract_majority_vote_label(cv19_dc_labeled)
cv19_ny_labeled = extract_majority_vote_label(cv19_ny_labeled)
cv19_ws_labeled = extract_majority_vote_label(cv19_ws_labeled)


In [ ]:
cv19_ny_labeled

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,retweeted,possibly_sensitive,lang,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries,Unnamed: 0,priority,categories
0,2020-05-03 14:57:57+00:00,1256961196525314048,1256961196525314048,"Reopening states will cause 233,000 more peopl...",False,"[0, 278]","{'hashtags': [{'text': 'NYC', 'indices': [212,...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,False,0.0,en,NaN,NaN,NaN,NaN,9758,Low,[Irrelevant]
1,2020-03-27 01:39:25+00:00,1243351890508701697,1243351890508701696,@samiglee @flash__delirium @michaelbd @mtracey...,False,"[47, 322]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,False,NaN,en,NaN,NaN,NaN,NaN,2807,Low,"[News, ThirdPartyObservation, Sentiment, Conte..."
2,2020-03-28 18:42:47+00:00,1243971816131104768,1243971816131104768,"We Met Up with Whalebone on Bleecker, and then...",False,"[0, 95]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",...,False,0.0,en,NaN,NaN,NaN,NaN,3369,Low,[Irrelevant]
3,2020-03-28 15:43:43+00:00,1243926751073513473,1243926751073513472,‘White-Collar Quarantine’ Over Virus Spotlight...,False,"[0, 123]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,False,1.0,en,NaN,NaN,NaN,NaN,2870,Low,[Irrelevant]
4,2020-03-29 03:17:45+00:00,1244101411899453441,1244101411899453440,The Lost Month: How a Failure to Test Blinded ...,False,"[0, 111]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",...,False,0.0,en,NaN,NaN,NaN,NaN,8537,Low,[Irrelevant]
5,2020-03-27 03:05:35+00:00,1243373573798010887,1243373573798010880,Coronavirus: Governors Ball 2020 cancelled ami...,False,"[0, 107]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1243373571554017286, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://publicize.wp.com/"" rel=""nofoll...",...,False,0.0,en,NaN,NaN,NaN,NaN,2633,High,"[News, Sentiment, ContextualInformation, Multi..."
6,2020-04-07 13:17:53+00:00,1247513929703661569,1247513929703661568,Some coronavirus positivity this morning. http...,False,"[0, 65]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,False,1.0,en,NaN,NaN,NaN,NaN,8281,Low,[Irrelevant]
7,2020-03-27 15:18:35+00:00,1243558040495960064,1243558040495960064,@MariaBartiromo @MorningsMaria @FoxBusiness A ...,False,"[44, 296]","{'hashtags': [{'text': 'Covid19', 'indices': [...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,False,NaN,en,NaN,NaN,NaN,NaN,2749,Low,"[Sentiment, ContextualInformation, Irrelevant]"
8,2020-03-27 09:28:35+00:00,1243469959017025536,1243469959017025536,» Correspondence from Dr Vladimir Zelenko on T...,False,"[0, 102]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",...,False,0.0,en,NaN,NaN,NaN,NaN,2755,Low,"[News, ContextualInformation, Irrelevant]"
9,2020-03-28 16:16:04+00:00,1243934894302146560,1243934894302146560,Just a reminder that #nopantssubwayride across...,False,"[0, 129]","{'hashtags': [{'text': 'nopantssubwayride', 'i...","{'media': [{'id': 1243934889143087104, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,False,0.0,en,NaN,NaN,NaN,NaN,

In [ ]:
def build_trainingset(trainSet):
  # Extract training set
  featureList = ['full_text','entities','favorite_count'] #,'favorited'
  trainF = EF(trainSet,featureList)

  labelList = ['priority']
  trainL_temp = EF(trainSet,labelList)['priority']
  trainF = Ehashtags(trainF)
  
  trainL = []
  for label in trainL_temp:
    if label == 'Low':
      trainL.append(0)
    elif label == 'Medium':
      trainL.append(1)
    elif label == 'High':
      trainL.append(2)
    elif label == 'Critical':
      trainL.append(3)
  return trainF, trainL

In [ ]:
def store_csv(df,location,val_ratio=0.2):
    trainF, trainL = build_trainingset(df)
    
    idx = np.arange(df.shape[0])
    np.random.shuffle(idx)
    
    val_size = int(len(idx) * val_ratio)
    if not os.path.exists('priority/'+location):
        os.makedirs('priority/'+location)
    
    print(len(trainL))
    df = pd.DataFrame(trainF)
    df['target'] = trainL
    df[['full_text','hashtags','favorite_count', 'target']].to_csv('priority/'+location+'/dataset_all.csv',
                   index=False)
    
    
    
    # df_tmp = df.iloc[idx[val_size:]
    # df_test = df.iloc[idx[:val_size]
    df.iloc[idx[val_size:], :][['full_text','hashtags','favorite_count','target']].to_csv('priority/'+location+'/dataset_test.csv',
                   index=False)
                      
    df.iloc[idx[val_size:], :][['full_text','hashtags','favorite_count','target']].to_csv(
        'priority/'+location+'/dataset_train.csv',index=False)
    
    # df.iloc[idx[:val_size], :][['full_text','entities','favorite_count', target]].to_csv(
    #     'cache/dataset_val.csv', index=False
    # )
    


In [ ]:
# Store training,  test set into csv file
if not os.path.exists('/priority/'):
  store_csv(cv19_ws_labeled,location='WS') 
  store_csv(cv19_dc_labeled,location='DC') 
  store_csv(cv19_ny_labeled,location='NY') 

2577
2504
2502


In [ ]:
# check GPU situation
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Aug 30 12:53:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:1A:00.0 Off |                  N/A |
| 41%   46C    P8    10W / 280W |      0MiB / 24190MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Pre-processing data set



In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 665kB 15.0MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 9.4MB/s eta 0:00:01
     |████████████████████████████████| 890kB 10.8MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 11.1MB/s eta 0:00:01
     |████████████████████████████████| 92kB 17.1MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bcbfd05d2b4b1597fe5d7207a7dd700ef2d65a2737f8f4c543f25d2dc356fcf5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
You should consider upgrading via the 'pip install --upgrade pip' command.


## Tokenization: Bert tokenizer

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

I0831 04:09:08.217187 139633676007232 file_utils.py:39] PyTorch version 1.6.0+cu101 available.
I0831 04:09:09.138132 139633676007232 file_utils.py:55] TensorFlow version 2.3.0 available.


Loading BERT tokenizer...


I0831 04:09:09.605131 139633676007232 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [ ]:
import pandas as pd
dataset_inputs = pd.read_csv('priority/NY/dataset_all.csv')

In [ ]:
dataset_inputs

,full_text,hashtags,favorite_count,target
0,"Reopening states will cause 233,000 more peopl...",NYC Georgia Florida Ohio California Michigan ...,0,0
1,@samiglee @flash__delirium @michaelbd @mtracey...,,0,0
2,"We Met Up with Whalebone on Bleecker, and then...",,22,0
3,‘White-Collar Quarantine’ Over Virus Spotlight...,,0,0
4,The Lost Month: How a Failure to Test Blinded ...,,0,0
5,Coronavirus: Governors Ball 2020 cancelled ami...,,0,2
6,Some coronavirus positivity this morning. http...,,0,0
7,@MariaBartiromo @MorningsMaria @FoxBusiness A ...,Covid19,0,0
8,» Correspondence from Dr Vladimir Zelenko on T...,,0,0
9,Just a reminder that #nopantssubwayride across...,nopantssubwayride COVID19 nyc subway Covid_19...,1,0


In [ ]:
sentences = dataset_inputs.full_text.values
labels = dataset_inputs.target.values

In [ ]:
for idx in range(len(sentences)):
  sentences[idx] = sentences[idx]  + ' '+ dataset_inputs.hashtags.values[idx] + ' ' + str(dataset_inputs.favorite_count.values[idx])

In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Reopening states will cause 233,000 more people to die from coronavirus, according to Wharton model: https://t.co/nYwBXFDVJR @realDonaldTrump @SpeakerPelosi @SenSchumer @AOC @RandPaul @SenSanders @RepKatiePorter #NYC #Georgia #Florida #Ohio #California #Michigan #Canada #London  NYC Georgia Florida Ohio California Michigan Canada London  0
Tokenized:  ['re', '##open', '##ing', 'states', 'will', 'cause', '233', ',', '000', 'more', 'people', 'to', 'die', 'from', 'corona', '##virus', ',', 'according', 'to', 'wharton', 'model', ':', 'https', ':', '/', '/', 't', '.', 'co', '/', 'ny', '##w', '##b', '##x', '##f', '##d', '##v', '##j', '##r', '@', 'real', '##don', '##ald', '##trum', '##p', '@', 'speaker', '##pel', '##osi', '@', 'sen', '##sch', '##ume', '##r', '@', 'ao', '##c', '@', 'rand', '##pa', '##ul', '@', 'sen', '##san', '##ders', '@', 'rep', '##kat', '##ie', '##port', '##er', '#', 'nyc', '#', 'georgia', '#', 'florida', '#', 'ohio', '#', 'california', '#', 'michigan', '#', 'can

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'


                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    # If the length of token is over the bert limitation, cut off the back
    if(len(encoded_sent)>144):
        diff = int((len(encoded_sent) - 145)/2)
        encoded_sent = encoded_sent[diff:144+diff]
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Reopening states will cause 233,000 more people to die from coronavirus, according to Wharton model: https://t.co/nYwBXFDVJR @realDonaldTrump @SpeakerPelosi @SenSchumer @AOC @RandPaul @SenSanders @RepKatiePorter #NYC #Georgia #Florida #Ohio #California #Michigan #Canada #London  NYC Georgia Florida Ohio California Michigan Canada London  0
Token IDs: [101, 2128, 26915, 2075, 2163, 2097, 3426, 22115, 1010, 2199, 2062, 2111, 2000, 3280, 2013, 21887, 23350, 1010, 2429, 2000, 24249, 2944, 1024, 16770, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 6396, 2860, 2497, 2595, 2546, 2094, 2615, 3501, 2099, 1030, 2613, 5280, 19058, 24456, 2361, 1030, 5882, 11880, 20049, 1030, 12411, 11624, 17897, 2099, 1030, 20118, 2278, 1030, 14566, 4502, 5313, 1030, 12411, 8791, 13375, 1030, 16360, 24498, 2666, 6442, 2121, 1001, 16392, 1001, 4108, 1001, 3516, 1001, 4058, 1001, 2662, 1001, 4174, 1001, 2710, 1001, 2414, 16392, 4108, 3516, 4058, 2662, 4174, 2710, 2414, 1014, 102]


In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  144


In [ ]:
import pandas as pd
tmp = pd.Series([len(sen) for sen in input_ids])

In [ ]:
# number of tweet which has over 144 tokens
np.sum(tmp>144)

0

## Padding


In [ ]:

# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.

MAX_LEN = 144

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 144 values...

Padding token: "[PAD]", ID: 0

Done.


Using TensorFlow backend.


## Attension mask

In [ ]:

# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

## Training & Validation Split



In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels, test_size=0.2)
# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels, test_size=0.2)

# Conduct the fine-tuning


In [ ]:
categories = {0:'Low',1:'Medium',2:'High',3:'Critical'}

## Search the best hyperparameters

In [ ]:
# build the test batch
testbatch = [test_inputs,test_labels,test_masks]

In [ ]:
from collections import defaultdict
all_metrics_search = defaultdict()
for batch_size in [16]: #16,
  for lr in [5e-5,3e-5,2e-5]:
    print('Currently processing %s'%(str(batch_size) + '_' + str(lr)))
    Bert = Bertnn(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,testbatch,batch_size=batch_size,lr=lr)
    all_metrics_one = Bert.searchUpsample(14)
    filename = str(batch_size) + '_' + str(lr)
    all_metrics_search[name] = all_metrics_one
    
    # store result into file
    with open(filename,'w') as file_obj:
      json.dump(all_metrics_one,file_obj)
      print('Successfully save file %s'%(filename))
    del Bert

Currently processing 16_5e-05
There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0827 09:27:57.691102 140115351164736 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0827 09:27:57.693360 140115351164736 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":

Current increase 0 times
Current is processing category 0
Before up-sample, there are 1600 samples
There are 1360 sampled in this category.
After up-sample, there are 1600 samples

Current is processing category 1
Before up-sample, there are 1600 samples
There are 47 sampled in this category.
After up-sample, there are 1600 samples

Current is processing category 2
Before up-sample, there are 1600 samples
There are 181 sampled in this category.
After up-sample, there are 1600 samples

Current is processing category 3
Before up-sample, there are 1600 samples
There are 12 sampled in this category.
After up-sample, there are 1600 samples

Currrent is processing on categorie Low

Before up-sample, the ratio of current category and all samples
1360
0.85

After up-sample, the ratio of current category and all samples
1360
0.85

Currrent is processing on categorie Medium

Before up-sample, the ratio of current category and all samples
47
0.029375

After up-sample, the ratio of current categor

I0827 09:28:01.846237 140115351164736 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0827 09:28:01.847458 140115351164736 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    100.    Elapsed: 0:00:17.
  Batch    80  of    100.    Elapsed: 0:00:34.

  Average training loss: 0.56
  Accuracy: 0.84
Category: 0
  Precision: 0.84
  Recall: 0.99
  F1: 0.91
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
Category: 2
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
Category: 3
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.2101
The average recall is: 0.2473
The average f1 is: 0.2267
  Training epcoh took: 0:00:43

Running Validation...
  Accuracy: 0.85
Category: 0
  Precision: 0.8486
  Recall: 1.0000
  F1: 0.9146
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
Category: 2
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
Category: 3
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.2121
The average recall is: 0.2500
The average f1 is: 0.2286
  Validation took: 0:00:03

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    100

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    265.    Elapsed: 0:00:19.
  Batch    80  of    265.    Elapsed: 0:00:37.
  Batch   120  of    265.    Elapsed: 0:00:56.
  Batch   160  of    265.    Elapsed: 0:01:15.
  Batch   200  of    265.    Elapsed: 0:01:35.
  Batch   240  of    265.    Elapsed: 0:01:54.

  Average training loss: 0.81
  Accuracy: 0.67
Category: 0
  Precision: 0.59
  Recall: 0.58
  F1: 0.55
Category: 1
  Precision: 0.17
  Recall: 0.12
  F1: 0.14
Category: 2
  Precision: 0.70
  Recall: 0.92
  F1: 0.78
Category: 3
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.3682
The average recall is: 0.4063
The average f1 is: 0.3659
  Training epcoh took: 0:02:05

Running Validation...
  Accuracy: 0.72
Category: 0
  Precision: 0.9211
  Recall: 0.7626
  F1: 0.8274
Category: 1
  Precision: 0.1538
  Recall: 0.1346
  F1: 0.1410
Category: 2
  Precision: 0.2234
  Recall: 0.5147
  F1: 0.2880
Category: 3
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.

I0827 14:18:37.821583 140115351164736 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0827 14:18:37.822580 140115351164736 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    280.    Elapsed: 0:00:19.
  Batch    80  of    280.    Elapsed: 0:00:38.
  Batch   120  of    280.    Elapsed: 0:00:57.
  Batch   160  of    280.    Elapsed: 0:01:16.
  Batch   200  of    280.    Elapsed: 0:01:35.
  Batch   240  of    280.    Elapsed: 0:01:54.

  Average training loss: 0.64
  Accuracy: 0.75
Category: 0
  Precision: 0.64
  Recall: 0.63
  F1: 0.61
Category: 1
  Precision: 0.53
  Recall: 0.46
  F1: 0.48
Category: 2
  Precision: 0.78
  Recall: 0.93
  F1: 0.84
Category: 3
  Precision: 0.03
  Recall: 0.03
  F1: 0.03
The average precision is: 0.4967
The average recall is: 0.5133
The average f1 is: 0.4870
  Training epcoh took: 0:02:13

Running Validation...
  Accuracy: 0.85
Category: 0
  Precision: 0.8837
  Recall: 0.9591
  F1: 0.9168
Category: 1
  Precision: 0.0385
  Recall: 0.0192
  F1: 0.0256
Category: 2
  Precision: 0.3333
  Recall: 0.3077
  F1: 0.2963
Category: 3
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.

I0827 14:27:45.625660 140115351164736 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0827 14:27:45.626669 140115351164736 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    295.    Elapsed: 0:00:19.
  Batch    80  of    295.    Elapsed: 0:00:38.
  Batch   120  of    295.    Elapsed: 0:00:57.
  Batch   160  of    295.    Elapsed: 0:01:16.
  Batch   200  of    295.    Elapsed: 0:01:35.
  Batch   240  of    295.    Elapsed: 0:01:54.
  Batch   280  of    295.    Elapsed: 0:02:13.

  Average training loss: 0.65
  Accuracy: 0.75
Category: 0
  Precision: 0.62
  Recall: 0.56
  F1: 0.56
Category: 1
  Precision: 0.54
  Recall: 0.48
  F1: 0.49
Category: 2
  Precision: 0.77
  Recall: 0.94
  F1: 0.83
Category: 3
  Precision: 0.10
  Recall: 0.09
  F1: 0.09
The average precision is: 0.5073
The average recall is: 0.5199
The average f1 is: 0.4951
  Training epcoh took: 0:02:20

Running Validation...
  Accuracy: 0.82
Category: 0
  Precision: 0.8995
  Recall: 0.9153
  F1: 0.9040
Category: 1
  Precision: 0.0385
  Recall: 0.0385
  F1: 0.0385
Category: 2
  Precision: 0.2615
  Recall: 0.3417
  F1: 0.2859
Category: 

In [ ]:
all_metrics_search

defaultdict(None,
            {'16_5e-05': {0: defaultdict(None,
                          {'train_loss_values': [0.5641749259829522,
                            0.5641749259829522,
                            0.5090744014829397,
                            0.5090744014829397,
                            0.3772193490900099,
                            0.3772193490900099,
                            0.24945555636659264,
                            0.24945555636659264],
                           'train_acc_all': [0.84, 0.85375, 0.890625, 0.9325],
                           'train_precision_all': [{0: 0.8405416666666667,
                             1: 0.0,
                             2: 0.0,
                             3: 0.0},
                            {0: 0.8588214285714286, 1: 0.0, 2: 0.1075, 3: 0.0},
                            {0: 0.9147370684870684,
                             1: 0.0,
                             2: 0.533047619047619,
                             3: 0.0},
   

## Incremental rectified training

In [ ]:
import json
batch_size = 32
lr = 5e-5
Bert = Bertnn(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,batch_size=batch_size,lr=lr)
all_metrics_one = Bert.searchUpsample(150)


del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0831 04:52:58.890795 139633676007232 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0831 04:52:58.891639 139633676007232 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":

Current increase 0 times
Current is processing category 0
Before up-sample, there are 1600 samples
There are 1351 sampled in this category.
After up-sample, there are 1600 samples

Current is processing category 1
Before up-sample, there are 1600 samples
There are 53 sampled in this category.
After up-sample, there are 1600 samples

Current is processing category 2
Before up-sample, there are 1600 samples
There are 182 sampled in this category.
After up-sample, there are 1600 samples

Current is processing category 3
Before up-sample, there are 1600 samples
There are 14 sampled in this category.
After up-sample, there are 1600 samples

Currrent is processing on categorie Low

Before up-sample, the ratio of current category and all samples
1351
0.844375

After up-sample, the ratio of current category and all samples
1351
0.844375

Currrent is processing on categorie Medium

Before up-sample, the ratio of current category and all samples
53
0.033125

After up-sample, the ratio of current

I0831 04:53:02.249113 139633676007232 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0831 04:53:02.249874 139633676007232 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":


======== Epoch 1 / 4 ========
Training...
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    40  of     50.    Elapsed: 0:00:15.
Invoke 3
Invoke 3
Invoke 3

  Average training loss: 0.97
  Accuracy: 0.83
Category: 0
  Precision: 0.85
  Recall: 0.98
  F1: 0.90
Category: 1
  Precision: 0.00
  Recall: 0.02
  F1: 0.00
Category: 2
  Precision: 0.00
  Recall: 0.01
  F1: 0.01
Category: 3
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.2133
The average recall is: 0.2541
The average f1 is: 0.2274
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.84
Category: 0
  Precision: 0.8355
  Recall: 1.0000
  F1: 0.9092
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
Category: 2
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
Category: 3
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.2089
The average recall is: 0.2500
The average f1 is: 0.2273
  Validation took: 0:00:

...


======== Epoch 1 / 4 ========
Training...
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    40  of  1,101.    Elapsed: 0:00:20.
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    80  of  1,101.    Elapsed: 0:00:40.
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch   120  of  1,101.    Elapsed: 0:01:00.
Invoke

I0831 14:05:56.373484 139633676007232 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0831 14:05:56.374389 139633676007232 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":


======== Epoch 1 / 4 ========
Training...
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    40  of  1,140.    Elapsed: 0:00:20.
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    80  of  1,140.    Elapsed: 0:00:39.
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3

I0831 14:42:54.602991 139633676007232 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0831 14:42:54.603943 139633676007232 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":


======== Epoch 1 / 4 ========
Training...
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    40  of  1,179.    Elapsed: 0:00:19.
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
  Batch    80  of  1,179.    Elapsed: 0:00:38.
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3
Invoke 3